### Split Apply Combine

In this notebook, we will work by grouping data together using what Pandas refers to as split-apply-combine. This lines up as follows:

- Split: split the data by grouping them on a feature or few features
- Apply: apply a mathematical aggregation or add another feature to each group
- Combine: combine these groups back together into one dataset

We will be using US housing data from King County, Washington (country of Seattle) to do so.

In [ ]:
import pandas as pd

In [ ]:
%pylab inline

In [ ]:
df = pd.read_csv('../data/kc_house_data.csv')

### Let's check our data and see if we have any cleanup

In [ ]:
df.head()

In [ ]:
df = df.set_index('id')

In [ ]:
df.head()

In [ ]:
df.dtypes

### Let's group by condition first, and see if we can use it to find any patterns

In [ ]:
df['condition'].value_counts()

In [ ]:
df.groupby('condition')

In [ ]:
for group, items in df.groupby('condition'):
    print(group)
    print(items.head())

In [ ]:
grped = df.groupby('condition')

In [ ]:
grped['price']

In [ ]:
grped['price'].mean()

In [ ]:
grped['price'].median()

In [ ]:
grped['price'].max() - grped['price'].min()

In [ ]:
grped['price'].hist(xrot=70, alpha=0.5)

## So now we know we have some serious outliers! For our purposes, let's remove them.

In [ ]:
df.price.describe()

In [ ]:
q1 = df.price.quantile(0.25)
q3 = df.price.quantile(0.75)

In [ ]:
print('first quartile:', q1)
print('third quartile:', q3)

In [ ]:
iqr = q3 - q1

In [ ]:
iqr

### Outlier via IQR from module two

- lower limit = q1 - 1.5 * iqr
- upper limit = q3 + 1.5 * iqr

In [ ]:
df.query('(@q1 - 1.5 * @iqr) <= price <= (@q3 + 1.5 * @iqr)')

In [ ]:
new_df = df.query('(@q1 - 1.5 * @iqr) <= price <= (@q3 + 1.5 * @iqr)')

In [ ]:
new_df.price.hist()

### Now our data looks much more manageable in terms of finding groups that affect price. How many rows did we remove? What do they look like?

In [ ]:
df.shape

In [ ]:
df.shape[0] - new_df.shape[0]

In [ ]:
df[~df.index.isin(new_df.index)]

In [ ]:
df[~df.index.isin(new_df.index)]['price'].max()

### Okay, so can we see now if condition affects price?

In [ ]:
grped = new_df.groupby('condition')

In [ ]:
grped.price.hist(alpha=0.5)

### Hmmm...
When we look at the above histogram, we don't  see obvious and extreme differences between groups. There are a few groups that are more evenly distributed and some that skew more to the lower end, but it doesn't look like a good predictor on price or that the groups have obvious different prices.

In [ ]:
grped = new_df.groupby('bedrooms')

In [ ]:
grped.indices.keys()

In [ ]:
grped.price.hist(stacked=True, alpha=0.2)

### This also doesn't appear to show a massive change, perhaps we can use corr to find some good potential features...

In [ ]:
new_df.corr()['price']

## I notice that the grade looks like it might be correlated with the price! let's have a look

In [ ]:
grped = new_df.groupby('grade')

In [ ]:
grped.price.hist(stacked=True, alpha=0.2)

In [ ]:
grped.price.mean()

In [ ]:
grped.price.median()

### Looks good, let's add an aggregation back to the table. This way we can see houses that stand out (much higher or lower cost than their group), as well as those that are more typical

In [ ]:
new_df.head()

In [ ]:
new_df.groupby('grade').price.transform('median')

In [ ]:
new_df['grade_median'] = new_df.groupby('grade').price.transform('median')

In [ ]:
new_df.groupby('grade').price.transform(lambda price: np.percentile(price, q=25))

In [ ]:
new_df['grade_1q'] = new_df.groupby('grade').price.transform(
    lambda price: np.percentile(price, q=25))

In [ ]:
new_df[new_df.price < new_df.grade_1q]

### Exercise: can you add the data for each grade's third quartile with regards to price? How many rows have a price above their grade's third quartile?

In [ ]:
# add code here

In [ ]:
%load ../solutions/04_quantile.py